In [1]:
import pandas as pd


In [2]:
df = pd.read_csv("spotify_millsongdata.csv")


In [3]:
df.head(5)


,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)


,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape


(57650, 4)

In [6]:
df.isnull().sum()


artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)


In [8]:
df.head(10)


,artist,song,text
0,America,Cornwall Blank,I've been down to the water \r\nI've been on ...
1,Snoop Dogg,Deep Cover,Tonight's the night I get in some shit [Yeah] ...
2,Quicksand,Slip,"Waist down, deep tied, now it's chest high. \..."
3,The Beatles,Old Brown Shoe,I want a love that's right but right is only h...
4,Kylie Minogue,Fine,"Fine, you're gonna be fine \r\nYou don't have..."
5,Coldplay,Oceans,"Wait for your call, love \r\nThe call never c..."
6,Gary Numan,Devotion,"Charm up, strange down \r\nOld face, new soun..."
7,Whitesnake,Hungry For Love,I keep hangin' on the promise of love \r\nYou...
8,Christmas Songs,Once Upon A Christmas,Once upon a Christmas far away in Bethlehem \...
9,Kenny Rogers,Me And Bobby McGee,"Busted flat, it baton rouge, headin' for the t..."


In [9]:
df['text'][0]


"I've been down to the water  \r\nI've been on top of the world  \r\nSomebody should come and see me  \r\nSomebody should give me the word  \r\n  \r\n'Cause I don't know how many more  \r\nI don't know how many more  \r\nI don't know how many more to see  \r\n  \r\nSometimes the free thoughts clash  \r\nSometimes I see through the grass  \r\nSometime the light begins to dim  \r\nSomebody should come and be me  \r\nSomebody should come and free me  \r\n  \r\n'Cause I don't know how many more (many more)  \r\nI don't know how many more (many more)  \r\nI don't know how many more to see  \r\n  \r\nOh darlin', darlin', I always want to see you  \r\nOh darlin', darlin', I always want to see you  \r\n  \r\nI cannot learn much more life  \r\nI cannot see much more  \r\nSomebody should stop the new sight  \r\nSomebody should close the door  \r\n  \r\n'Cause I don't know how many more (many more)  \r\nI don't know how many more (many more)  \r\nI don't know how many more to see  \r\n  \r\nWrapp

In [10]:
df.shape


(5000, 3)

In [11]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)


In [16]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [ ]:
df['text'] = df['text'].apply(lambda x: tokenization(x))


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [22]:
similarity[0]


array([1.        , 0.13429967, 0.02880037, ..., 0.01855084, 0.00593757,
       0.0440162 ])

In [23]:
df[df['song'] == 'Crying Over You']


,artist,song,text
2439,ABBA,Crying Over You,i'm waitin' for you baby \r i'm sitting all a...


In [24]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])

    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)

    return songs

In [ ]:
recommendation('Crying Over You')


In [26]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))